In [38]:
# sys
import os
import uuid

# langchain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader

# chroma
import chromadb
from chromadb.utils import embedding_functions
from chromadb.config import Settings

OPEN_API_KEY = "asdf"

In [39]:
loader = TextLoader('sotu.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

documents = []
metadatas = []
ids = []
for doc in docs:
    documents.append(doc.page_content)
    metadatas.append(doc.metadata)
    ids.append(str(uuid.uuid4()))

In [40]:
client = chromadb.Client(Settings(
    chroma_db_impl="duckdb+parquet",
    persist_directory="./SentenceTransformers",
))
client.reset()

openai_ef = embedding_functions.OpenAIEmbeddingFunction(
                api_key=OPEN_API_KEY,
                model_name="text-embedding-ada-002"
            )

collection = client.create_collection(name="sotu")

collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

client.persist()

No embedding_function provided, using default embedding function: DefaultEmbeddingFunction https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2


True